# State Farm

In [74]:
%matplotlib inline
from __future__ import division,print_function

import os, json
from shutil import copyfile
from glob import glob
import numpy as np
import pandas as pd
import re
from keras.utils.data_utils import get_file
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt
from PIL import Image
import keras
from keras.optimizers import SGD, Adam
from keras.applications.vgg16 import VGG16
from keras.models import Model, Sequential
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, Dropout, Flatten, Input, BatchNormalization, Convolution2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras_tqdm import TQDMNotebookCallback
from utils import plots, get_batches, plot_confusion_matrix, get_data


## Load data

## Create validation set

Important that the validation set not only contains a random selection of each class but also for each person, since there is multiple images of each person

In [118]:
path = "data/statefarm/"
#path = "data/statefarm/sample/"

In [3]:
model_path = path + 'models/'
if not os.path.exists(model_path): os.mkdir(model_path)
batch_size=64

Read csv file and create set of person and classes and images

In [43]:
valid_path = path + 'valid/'
if not os.path.exists(valid_path): os.mkdir(valid_path) 
person_dict = {}
with open(path + 'driver_imgs_list.csv') as f:
    for line in f:
        words = line.rstrip().split(',')
        person_id = words[0]
        label = words[1]
        img = words[2]
        
        if person_id not in person_dict:
            person_dict[person_id] = {}
            person_dict[person_id][label] = [img]    
        elif label not in person_dict[person_id]:
            person_dict[person_id][label] = [img]
        else: 
            person_dict[person_id][label].append(img)
            


In [ ]:
for p in person_dict.keys():
    for c in person_dict[p].keys():
        g = person_dict[p][c]
        random_order_images = np.random.permutation(g)
        if not os.path.exists(valid_path + c): os.mkdir(valid_path + c) 
        for i in range(int(len(random_order_images) * 0.1)):
            os.rename((path + "train/" + c + "/" + random_order_images[i]), (valid_path + c + "/" + random_order_images[i]))


Create a sample set with 50 picture of training images and 10 validation images

In [ ]:
for p in person_dict.keys():
    for c in person_dict[p].keys():
        g = person_dict[p][c]
        random_order_images = np.random.permutation(g)
        if not os.path.exists(valid_path + c): os.mkdir(valid_path + c) 
        for i in range(int(len(random_order_images) * 0.1)):
            os.rename((path + "train/" + c + "/" + random_order_images[i]), (valid_path + c + "/" + random_order_images[i]))

In [56]:
sample_path = path + 'sample/'
sample_valid_path = sample_path + 'valid/'
sample_train_path = sample_path + 'train/'
if not os.path.exists(sample_path): os.mkdir(sample_path) 
if not os.path.exists(sample_valid_path): os.mkdir(sample_valid_path)
if not os.path.exists(sample_train_path): os.mkdir(sample_train_path) 
for p in person_dict.keys():
    for c in person_dict[p].keys():
        if not os.path.exists(sample_train_path + c): os.mkdir(sample_train_path + c)
        if not os.path.exists(sample_valid_path + c): os.mkdir(sample_valid_path + c) 
        try:
            train_files_in_class = os.listdir(path + "train/" + c) 
            valid_files_in_cass = os.listdir(valid_path + c)
            for i in range(140):
                copyfile((path + "train/" + c + "/" + train_files_in_class[i]), (sample_train_path + c + "/" + train_files_in_class[i]))
            for i in range(140, 180):
                copyfile((path + "valid/" + c + "/" + valid_files_in_cass[i]), (sample_valid_path + c + "/" + valid_files_in_cass[i]))
        except:
            print("Something went wrong")
    

Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong
Something went wrong


### Load data and fit to net

In [12]:
im_size=224

In [106]:
train_gen = ImageDataGenerator().flow_from_directory(path + "train",
                                              batch_size=batch_size,
                                              class_mode='categorical',
                                              target_size=(224, 224),
                                              shuffle=True);
val_gen = ImageDataGenerator().flow_from_directory(path + "valid",
                                          batch_size=batch_size*2,
                                          class_mode='categorical',
                                          target_size=(224, 224),
                                          shuffle=False)
num_class=len(train_gen.class_indices)


Found 1417 images belonging to 10 classes.
Found 898 images belonging to 10 classes.


### Creating a linear model

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
flatten_4 (Flatten)              (None, 150528)        0           input_8[0][0]                    
____________________________________________________________________________________________________
dense_10 (Dense)                 (None, 10)            1505290     flatten_4[0][0]                  
Total params: 1505290
____________________________________________________________________________________________________


In [31]:
inputs = Input(shape=(3, im_size, im_size))
x = BatchNormalization(inputs)
x = Flatten()(inputs)
x = Dense(num_class, activation='softmax')(x)

model = Model(input=inputs, output=x)

model.compile(Adam(lr=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

### Fit linear model

In [60]:
nb_epoch=1

In [61]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.N, nb_epoch=nb_epoch, 
                        validation_data=val_gen, nb_val_samples=val_gen.N, callbacks=[TQDMNotebookCallback()])

Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


Epoch 1/1
1417/1417 [==============================] - 33s - loss: 14.5256 - acc: 0.0988 - val_loss: 14.5027 - val_acc: 0.1002


In [62]:
model.optimizer.lr=0.001

In [64]:
model.fit_generator(train_gen, samples_per_epoch=train_gen.N, nb_epoch=nb_epoch, 
                        validation_data=val_gen, nb_val_samples=val_gen.N, callbacks=[TQDMNotebookCallback()])

Epoch 1/1


Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"
Widget Javascript not detected.  It may not be installed properly. Did you enable the widgetsnbextension? If not, then run "jupyter nbextension enable --py --sys-prefix widgetsnbextension"


1417/1417 [==============================] - 33s - loss: 14.5256 - acc: 0.0988 - val_loss: 14.5027 - val_acc: 0.1002


### A super simple CNN network

In [111]:
#Simple VGG inspired CNN
inputs = Input(shape=(3, im_size, im_size))
x = BatchNormalization(axis=1)(inputs)
x = Convolution2D(64, 3, 3, activation='relu', name='block1_conv1')(x)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D((3, 3), name='block1_pool')(x)
x = Convolution2D(64, 3, 3, activation='relu', name='block1_conv2')(x)
x = BatchNormalization(axis=1)(x)
x = MaxPooling2D((3, 3), name='block2_pool')(x)
x = Flatten()(x)
x = Dense(2000, activation='relu')(x)
x = BatchNormalization(axis=1)(x)
x = Dense(num_class, activation='softmax')(x)

cnn = Model(input=inputs, output=x)

cnn.compile(Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

cnn.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_27 (InputLayer)            (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
batchnormalization_56 (BatchNorma(None, 3, 224, 224)   6           input_27[0][0]                   
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 222, 222)  1792        batchnormalization_56[0][0]      
____________________________________________________________________________________________________
batchnormalization_57 (BatchNorma(None, 64, 222, 222)  128         block1_conv1[0][0]               
___________________________________________________________________________________________

In [112]:
nb_epoch=2
cnn.fit_generator(train_gen, samples_per_epoch=train_gen.N, nb_epoch=nb_epoch, 
                        validation_data=val_gen, nb_val_samples=val_gen.N)

Epoch 1/2
1417/1417 [==============================] - 370s - loss: 2.0892 - acc: 0.5794 - val_loss: 2.2792 - val_acc: 0.2450
Epoch 2/2
1417/1417 [==============================] - 362s - loss: 0.2547 - acc: 0.9323 - val_loss: 1.4963 - val_acc: 0.4800


In [113]:
cnn.optimizer.lr=0.001
nb_epoch=2
cnn.fit_generator(train_gen, samples_per_epoch=train_gen.N, nb_epoch=nb_epoch, 
                        validation_data=val_gen, nb_val_samples=val_gen.N)

Epoch 1/10
1417/1417 [==============================] - 371s - loss: 0.0590 - acc: 0.9873 - val_loss: 1.6755 - val_acc: 0.4098
Epoch 2/10
1417/1417 [==============================] - 363s - loss: 0.0222 - acc: 0.9979 - val_loss: 1.7229 - val_acc: 0.4566
Epoch 3/10
 192/1417 [===>..........................] - ETA: 255s - loss: 0.0111 - acc: 1.0000

KeyboardInterrupt: 

### Load weights

In [115]:
weigth_file_name =  model_path + 'simple-cnn-%d-one-epoch.h5'

In [67]:
model.load_weights(weigth_file_name)

IOError: Unable to open file (Unable to open file: name = 'data/statefarm/models/weights–ft-vgg16-dogs%d.h5', errno = 2, error message = 'no such file or directory', flags = 0, o_flags = 0)

### Save weights

In [120]:
model.save_weights(weigth_file_name)

'data/statefarm/'

# Run on test data

In [ ]:
print(path + "test")
test_gen = ImageDataGenerator().flow_from_directory(path + "test",
                                          batch_size=batch_size*2,
                                          class_mode=None,
                                          target_size=(224, 224),
                                          shuffle=False)
test_predictions = cnn.predict_generator(test_gen, test_gen.nb_sample)

data/statefarm/test
Found 79725 images belonging to 1 classes.


In [ ]:
ids = list(map(lambda x: [re.search('.+\/(.+)\.jpg', x).group(1)], test_gen.filenames))
subm = np.hstack((ids, test_predictions))
class_str = ','.join(['id'] + cnn.classes)
N = len(cnn.classes)
format_str = ','.join(['%s'] + ['%s']*N)
submission_file_name = 'statefarm.csv'
np.savetxt(submission_file_name, subm, fmt=format_str, header=class_str, comments='')

### Downloadable link

In [ ]:
from IPython.display import FileLink
submission_file_name = 'statefarm.csv'
FileLink(submission_file_name)